In [44]:
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import re

# Load the dataset
data = pd.read_csv('output_new.csv')

# Data Cleaning
data['team_1'] = data['team_1'].fillna('Unknown')
data['team_2'] = data['team_2'].fillna('Unknown')
data['pom'] = data['pom'].fillna('No Award')
data['world_cup_year'] = data['world_cup_year'].astype(str)

def extract_winner(result):
    match = re.search(r'\b([A-Za-z\s]+)\s+won\b', result)
    if match:
        return match.group(1).strip() 
    return None  

# Initialize Dash app
app = Dash(__name__)

# App Layout
app.layout = html.Div([
    html.H1("ICC Men’s Cricket World Cup - A Journey Through History ",style={'fontSize':48,'color':'darkblue','textAlign': 'center'}),

    html.Div([
        dcc.Dropdown(
            id='match_category_filter',
            options=[{'label': cat, 'value': cat} for cat in data['match_category'].unique()],
            placeholder='Select Match Category',
            style={'width': '300px'}
        ),
        dcc.Dropdown(
            id='year_filter',
            options=[{'label': year, 'value': year} for year in data['world_cup_year'].unique()],
            placeholder='Select Year',
            style={'width': '300px'}
        ),
    ], style={'display': 'flex','flexDirection':'column','alignItems':'center','gap': '10px', 'marginBottom': '20px'}),
    html.Div([
        dcc.Graph(id='pie_chart_matches'),
        dcc.Graph(id='bar_chart_runs'),
        dcc.Graph(id='bar_chart_wickets'),
        dcc.Graph(id='bar_chart_mom'),
    ])
])

# Callback to update graphs
@app.callback(
    [
        Output('pie_chart_matches', 'figure'),
        Output('bar_chart_runs', 'figure'),
        Output('bar_chart_wickets', 'figure'),
        Output('bar_chart_mom', 'figure')
    ],
    [
        Input('match_category_filter', 'value'),
        Input('year_filter', 'value')
    ]
)

def update_charts(match_category, year):
    # Filter data based on dropdown selections
    filtered_data = data.copy()
    if match_category:
        filtered_data = filtered_data[filtered_data['match_category'] == match_category].copy()
    if year:
        filtered_data = filtered_data[filtered_data['world_cup_year'] == year].copy()

    # Pie Chart: Total Matches Won by Team
    filtered_data['winner'] = filtered_data['result'].apply(extract_winner)
    filtered_data['winner'] = filtered_data['winner'].dropna()
    matches_won = filtered_data['winner'].value_counts().reset_index()
    matches_won.columns = ['Team', 'Matches Won']
    pie_chart = px.pie(matches_won, names='Team', values='Matches Won', title='Total Matches Won by Team')
    
    # Bar Chart: Total Runs by Team
    total_runs_team_1 = filtered_data.groupby('team_1')['team_1_runs'].sum().reset_index()
    total_runs_team_2 = filtered_data.groupby('team_2')['team_2_runs'].sum().reset_index()
    total_runs_team_1.columns = ['Team', 'Runs']
    total_runs_team_2.columns = ['Team', 'Runs']
    total_runs = pd.concat([total_runs_team_1, total_runs_team_2])
    total_runs = total_runs.groupby('Team')['Runs'].sum().reset_index()
    bar_chart_runs = px.bar(total_runs, x='Team', y='Runs', title='Total Runs by Team', text='Runs')
    
    # Bar Chart: Total Wickets by Team
    total_wickets_team_1 = filtered_data.groupby('team_1')['team_1_wickets'].sum().reset_index()
    total_wickets_team_2 = filtered_data.groupby('team_2')['team_2_wickets'].sum().reset_index()
    total_wickets_team_1.columns = ['Team', 'Wickets']
    total_wickets_team_2.columns = ['Team', 'Wickets']
    total_wickets = pd.concat([total_wickets_team_1, total_wickets_team_2])
    total_wickets = total_wickets.groupby('Team')['Wickets'].sum().reset_index()
    bar_chart_wickets = px.bar(total_wickets, x='Team', y='Wickets', title='Total Wickets by Team', text='Wickets')
    
    # Bar Chart: Most Valuable Player
    mom_counts = filtered_data[filtered_data['pom'] != 'No Award']['pom'].value_counts().head(10).reset_index()
    mom_counts.columns = ['Player', 'POM Awards']
    bar_chart_mom = px.bar(mom_counts, x='Player', y='POM Awards', title='Most Valuable Players (POM Awards)', text='POM Awards')
    
    return pie_chart, bar_chart_runs, bar_chart_wickets, bar_chart_mom

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)